In [ ]:
# Utilities
import os
import csv
import argparse
from time import time
# Keras
import tensorflow as tf
from sklearn.metrics import silhouette_score, pairwise_distances
from keras.models import Model
from keras.layers import Dense, Reshape, UpSampling2D, Conv2DTranspose, GlobalAveragePooling1D, Softmax
from keras.losses import kullback_leibler_divergence
import keras.backend as K
# scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans
# Dataset helper function
# DTC components
from keras.models import Model
from keras.layers import Input, Conv1D, LeakyReLU, MaxPool1D, LSTM, Bidirectional, TimeDistributed, Dense, Reshape
from keras.layers import UpSampling2D, Conv2DTranspose
import numpy as np
from sklearn import metrics
from scipy.optimize import linear_sum_assignment
from sklearn.preprocessing import label_binarize
from tensorflow.keras.layers import Layer, InputSpec
import keras.backend as K
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, Attention
from tensorflow.keras import layers
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
import pickle

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow_addons as tfa
from math import pi, ceil

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from keras.losses import CategoricalCrossentropy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
# %tensorflow_version 1.x
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time
print(tf.__version__)
from keras.layers import MultiHeadAttention
from keras.layers import Dense
import gc
from keras.layers import concatenate
import csv
import math
import warnings
import os

# import xgboost as xgb
warnings.filterwarnings('ignore')
# import GPy, GPyOpt
tfkl = tf.keras.layers
tfk = tf.keras

from texttable import Texttable

In [ ]:
import time

In [ ]:
from tensorflow.keras.losses import KLDivergence

In [ ]:
def autoencoder(latent_dim=8, series_len=None):

    # Input
    x = Input((None, series_len),name='input')
    #masked_seq = layers.Masking(mask_value=0.0)(encoder_inputs)
    # Encoder
    encoded = LSTM(256, return_sequences=True, name="encoded_lstm1")(x)
    encoded = LSTM(128, return_sequences=True, name="encoded_lstm2")(encoded)
    attention = Attention(name="encoded_atten")([encoded, encoded, encoded])  # Apply self-attention to the encoder outputs
    merged = layers.Concatenate(axis=-1)([encoded, attention])        
    encoded = LSTM(64, name="encoded_lstm3")(merged)
    encoded = Dense(latent_dim, name="encoded_dense")(encoded)

    # Decoder
    decoded = RepeatVector(1)(encoded)
    decoded = LSTM(64, return_sequences=True, name="decoded_lstm1")(decoded)
    decoded = LSTM(128, return_sequences=True, name="decoded_lstm2")(decoded)
    decoded = LSTM(256, return_sequences=True, name="decoded_lstm3")(decoded)
    attention = Attention(name="decoded_atten")([decoded, decoded, decoded])  # Apply self-attention to the decoder outputs
    decoded = layers.Concatenate(axis=-1, name="decoded_concat")([decoded, attention])
    decoder_outputs = TimeDistributed(Dense(series_len), name="decoded_timeD")(decoded)
    
    # AE model
    autoencoder = Model(inputs=x, outputs=decoder_outputs, name='AE')

    # Encoder model
    encoder = Model(inputs=x, outputs=encoded, name='encoder')

    # Create input for decoder model
    encoded_input =Input(shape=(latent_dim,))
    # Internal layers in decoder
    decoded = RepeatVector(1)(encoded_input)
    decoded = autoencoder.get_layer('decoded_lstm1')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm2')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm3')(decoded)
    attention = autoencoder.get_layer('decoded_atten')([decoded, decoded, decoded])
    decoded = autoencoder.get_layer('decoded_concat')([decoded, attention])
    decoder_output = autoencoder.get_layer('decoded_timeD')(decoded)
    # Decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_output, name='decoder')

    return autoencoder, encoder, decoder

In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras import models

def autoencoder_NN(series_len=270, latent_dim=10):
    """
    Creates the encoder and decoder for the autoencoder model with shape (None, latent_dim).
    """
    # Encoder
    encoder_input = Input((series_len,), name='input')
    x = Dense(500, activation='relu', name='encode_FC1_500')(encoder_input)
    x = Dense(500, activation='relu', name='encode_FC2_500')(x)
    x = Dense(2000, activation='relu', name='encode_FC3_2000')(x)
    latent_space = Dense(latent_dim, name="latent_space")(x)

    encoder = models.Model(inputs=encoder_input, outputs=latent_space, name="Encoder")

    # Decoder
    decoder_input = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(2000, activation='relu', name='decode_FC1_2000')(decoder_input)
    x = Dense(500, activation='relu', name='decode_FC2_500')(x)
    x = Dense(500, activation='relu', name='decode_FC3_500')(x)

    # Add a time dimension
    x = Dense(series_len, activation=None, name="reconstructed_output")(x)

    decoder = models.Model(inputs=decoder_input, outputs=x, name="Decoder")

    # Autoencoder
    encoded = encoder(encoder_input)
    decoded = decoder(encoded)

    autoencoder = models.Model(inputs=encoder_input, outputs=decoded, name="Autoencoder")

    return autoencoder, encoder, decoder

In [ ]:
def autoencoderNeighbour(latent_dim=8, series_len=None):

    # Input
    x = Input((None, series_len), name='input')
    #masked_seq = layers.Masking(mask_value=0.0)(encoder_inputs)
    # Encoder
    encoded = LSTM(256, return_sequences=True, name='lstm_input_256')(x)
    encoded = LSTM(128, return_sequences=True, name='lstm_input_128')(encoded)
    attention = Attention(name='attention_input')([encoded, encoded, encoded])  # Apply self-attention to the encoder outputs
    merged = layers.Concatenate(axis=-1)([encoded, attention])        
    encoded = LSTM(64, name='lstm_input_64')(merged)
    encoded = Dense(latent_dim, name='dense_input')(encoded)
    
    
    
    
    x_neighbour1 = Input((None, series_len), name='Neighbour1_input')
    x_neighbour2 = Input((None, series_len), name='Neighbour2_input')
    
    encoded_neighbour1 = LSTM(256, return_sequences=True, name='lstm_Neighbour1_256')(x_neighbour1)
    encoded_neighbour2 = LSTM(256, return_sequences=True, name='lstm_Neighbour2_256')(x_neighbour2)
    encoded_neighbour1 = LSTM(128, return_sequences=True, name='lstm_Neighbour1_128')(encoded_neighbour1)
    encoded_neighbour2 = LSTM(128, return_sequences=True, name='lstm_Neighbour2_128')(encoded_neighbour2)
    attention_neighbour1 = Attention(name='attention_Neighbour1')([encoded_neighbour1, encoded_neighbour2, encoded_neighbour2])  # Apply self-attention to the encoder outputs
    attention_neighbour2 = Attention(name='attention_Neighbour2')([encoded_neighbour2, encoded_neighbour1, encoded_neighbour1])  # Apply self-attention to the encoder outputs
    merged_neighbour1 = layers.Concatenate(axis=-1)([encoded_neighbour1, attention_neighbour1])        
    merged_neighbour2 = layers.Concatenate(axis=-1)([encoded_neighbour2, attention_neighbour2])        
    encoded_neighbour1 = LSTM(64, name='lstm_Neighbour1_64')(merged_neighbour1)
    encoded_neighbour2 = LSTM(64, name='lstm_Neighbour2_64')(merged_neighbour2)
    encoded_neighbour1 = Dense(latent_dim, name='dense_Neighbour1')(encoded_neighbour1)
    encoded_neighbour2 = Dense(latent_dim, name='dense_Neighbour2')(encoded_neighbour2)

    out_encoder_Neighbor = layers.Maximum()([encoded_neighbour1, encoded_neighbour2]) 
    # add Final encoder
    encoded = layers.add([encoded ,out_encoder_Neighbor],name='add_inputs')
    
    
    # Decoder
    decoded = RepeatVector(1)(encoded)
    decoded = LSTM(64, return_sequences=True, name="decoded_lstm1")(decoded)
    decoded = LSTM(128, return_sequences=True, name="decoded_lstm2")(decoded)
    decoded = LSTM(256, return_sequences=True, name="decoded_lstm3")(decoded)
    attention = Attention(name="decoded_atten")([decoded, decoded, decoded])  # Apply self-attention to the decoder outputs
    decoded = layers.Concatenate(axis=-1, name="decoded_concat")([decoded, attention])
    decoder_outputs = TimeDistributed(Dense(series_len), name="decoded_timeD")(decoded)
    
    # AE model
    autoencoder = Model(inputs=[x,x_neighbour1,x_neighbour2], outputs=decoder_outputs, name='AE')

    # Encoder model
    encoder = Model(inputs=[x,x_neighbour1,x_neighbour2], outputs=encoded, name='encoder')

    # Create input for decoder model
    encoded_input =Input(shape=(latent_dim,))
    # Internal layers in decoder
    decoded = RepeatVector(1)(encoded_input)
    decoded = autoencoder.get_layer('decoded_lstm1')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm2')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm3')(decoded)
    attention = autoencoder.get_layer('decoded_atten')([decoded, decoded, decoded])
    decoded = autoencoder.get_layer('decoded_concat')([decoded, attention])
    decoder_output = autoencoder.get_layer('decoded_timeD')(decoded)
    # Decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_output, name='decoder')

    return autoencoder, encoder, decoder

In [ ]:
def autoencoder_NN_Neighbour(latent_dim=10, series_len=None):

    # Input
    encoder_input = Input((series_len,), name='input')
    x = Dense(500, activation='relu', name='encode_FC1_500')(encoder_input)
    x = Dense(500, activation='relu', name='encode_FC2_500')(x)
    x = Dense(2000, activation='relu', name='encode_FC3_2000')(x)
    latent_space = Dense(latent_dim, name="latent_space")(x)
    
    
    
    
    x_neighbour1 = Input((series_len,), name='Neighbour1_input')
    x_neighbour2 = Input((series_len,), name='Neighbour2_input')
    
    encoded_neighbour1 = Dense(500, activation='relu', name='encode_Neighbour1_FC1_500')(x_neighbour1)
    encoded_neighbour2 = Dense(500, activation='relu', name='encode_Neighbour2_FC1_500')(x_neighbour2)
    encoded_neighbour1 = Dense(500, activation='relu', name='encode_Neighbour1_FC2_500')(encoded_neighbour1)
    encoded_neighbour2 = Dense(500, activation='relu', name='encode_Neighbour2_FC2_500')(encoded_neighbour2)        
    encoded_neighbour1 = Dense(2000, activation='relu', name='encode_Neighbour1_FC3_2000')(encoded_neighbour1)
    encoded_neighbour2 = Dense(2000, activation='relu', name='encode_Neighbour2_FC3_2000')(encoded_neighbour2)
    encoded_neighbour1 = Dense(latent_dim, name="latent_Neighbour1_space")(encoded_neighbour1)
    encoded_neighbour2 = Dense(latent_dim, name="latent_Neighbour2_space")(encoded_neighbour2)

    out_encoder_Neighbor = layers.Maximum()([encoded_neighbour1, encoded_neighbour2]) 
    # add Final encoder
    encoded = layers.add([latent_space ,out_encoder_Neighbor],name='add_inputs')
    
    
    # Decoder
    decoder_input = Input(shape=(latent_dim,), name='decoder_input')
    x = Dense(2000, activation='relu', name='decode_FC1_2000')(decoder_input)
    x = Dense(500, activation='relu', name='decode_FC2_500')(x)
    x = Dense(500, activation='relu', name='decode_FC3_500')(x)
    # Add a time dimension
    x = Dense(series_len, activation=None, name="reconstructed_output")(x)

    
    encoder = models.Model(inputs=[encoder_input,x_neighbour1,x_neighbour2], outputs=encoded, name="Encoder")
    decoder = models.Model(inputs=decoder_input, outputs=x, name="Decoder")
    encoded = encoder([encoder_input, x_neighbour1, x_neighbour2])
    decoded = decoder(encoded)
    autoencoder = models.Model(inputs=[encoder_input,x_neighbour1,x_neighbour2], outputs=decoded, name="Autoencoder")
    return autoencoder, encoder, decoder

In [ ]:
def eucl(x, y):
    """
    Euclidean distance between two multivariate time series given as arrays of shape (timesteps, dim)
    """
    d = np.sqrt(np.sum(np.square(x - y), axis=0))
    return np.sum(d)


def cid(x, y):
    """
    Complexity-Invariant Distance (CID) between two multivariate time series given as arrays of shape (timesteps, dim)
    Reference: Batista, Wang & Keogh (2011). A Complexity-Invariant Distance Measure for Time Series. https://doi.org/10.1137/1.9781611972818.60
    """
    assert(len(x.shape) == 2 and x.shape == y.shape)  # time series must have same length and dimensionality
    ce_x = np.sqrt(np.sum(np.square(np.diff(x, axis=0)), axis=0) + 1e-9)
    ce_y = np.sqrt(np.sum(np.square(np.diff(y, axis=0)), axis=0) + 1e-9)
    d = np.sqrt(np.sum(np.square(x - y), axis=0)) * np.divide(np.maximum(ce_x, ce_y), np.minimum(ce_x, ce_y))
    return np.sum(d)


def cor(x, y):
    """
    Correlation-based distance (COR) between two multivariate time series given as arrays of shape (timesteps, dim)
    """
    scaler = TimeSeriesScalerMeanVariance()
    x_norm = scaler.fit_transform(x)
    y_norm = scaler.fit_transform(y)
    pcc = np.mean(x_norm * y_norm)  # Pearson correlation coefficients
    d = np.sqrt(2.0 * (1.0 - pcc + 1e-9))  # correlation-based similarities
    return np.sum(d)


def acf(x, y):
    """
    Autocorrelation-based distance (ACF) between two multivariate time series given as arrays of shape (timesteps, dim)
    Computes a linearly weighted euclidean distance between the autocorrelation coefficients of the input time series.
    Reference: Galeano & Pena (2000). Multivariate Analysis in Vector Time Series.
    """
    assert (len(x.shape) == 2 and x.shape == y.shape)  # time series must have same length and dimensionality
    x_acf = np.apply_along_axis(lambda z: stattools.acf(z, nlags=z.shape[0]), 0, x)
    y_acf = np.apply_along_axis(lambda z: stattools.acf(z, nlags=z.shape[0]), 0, y)
    weights = np.linspace(1.0, 0.0, x.shape[0])
    d = np.sqrt(np.sum(np.expand_dims(weights, axis=1) * np.square(x_acf - y_acf), axis=0))
    return np.sum(d)

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import precision_score, recall_score, adjusted_rand_score, fowlkes_mallows_score

def cluster_metrics(y_true, y_pred):
    """
    Calculate unsupervised clustering accuracy, precision, recall, Rand Index, and Fowlkes–Mallows Index.

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        A dictionary with accuracy, precision, recall, Rand Index, and FMI.
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    
    # Determine the mapping using Hungarian algorithm
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    
    # Map predicted labels to true labels based on the Hungarian algorithm
    label_mapping = {row: col for row, col in zip(row_ind, col_ind)}
    y_pred_mapped = np.array([label_mapping[label] for label in y_pred])
    
    # Calculate accuracy
    accuracy = w[row_ind, col_ind].sum() * 1.0 / y_pred.size
    
    # Calculate precision and recall
    precision = precision_score(y_true, y_pred_mapped, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred_mapped, average='weighted', zero_division=0)
    
    # Calculate Rand Index
    rand_index = adjusted_rand_score(y_true, y_pred)
    
    # Calculate Fowlkes–Mallows Index
    fmi = fowlkes_mallows_score(y_true, y_pred)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'rand_index': rand_index,
        'fowlkes_mallows_index': fmi
    }

def cluster_acc(y_true, y_pred):
    """
    Calculate unsupervised clustering accuracy. Requires scikit-learn installed

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return w[row_ind, col_ind].sum() * 1.0 / y_pred.size


def cluster_purity(y_true, y_pred):
    """
    Calculate clustering purity

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        purity, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    label_mapping = w.argmax(axis=1)
    y_pred_voted = y_pred.copy()
    for i in range(y_pred.size):
        y_pred_voted[i] = label_mapping[y_pred[i]]
    return metrics.accuracy_score(y_pred_voted, y_true)


def roc_auc(y_true, q_pred, n_classes):
    """
    Calculate area under ROC curve (ROC AUC)
    WARNING: DO NOT USE, MAY CONTAIN ERRORS
    TODO: CHECK IT!

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        q_pred: predicted probabilities, numpy.array with shape `(n_samples,)`

    # Return
        ROC AUC score, in [0,1]
    """
    if n_classes == 2:  # binary ROC AUC
        auc = max(metrics.roc_auc_score(y_true, q_pred[:, 1]), metrics.roc_auc_score(y_true, q_pred[:, 0]))
    else:  # micro-averaged ROC AUC (multiclass)
        fpr, tpr, _ = metrics.roc_curve(label_binarize(y_true, classes=np.unique(y_true)).ravel(), q_pred.ravel())
        auc = metrics.auc(fpr, tpr)
    return auc

In [ ]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, FeatureWeight, weights=None, a=1.0, b=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.FeatureWeight = FeatureWeight
        self.a = a
        self.b = b
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """

        global FeatureWeight
        q = (1 + self.a * (K.sum((self.FeatureWeight) * K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) ** (self.b))) ** (-1)  
        FeatureWeight = K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters) * K.expand_dims(q**2, axis=2)  ,axis=0)
        FeatureWeight = K.transpose(K.transpose(FeatureWeight) / K.sum(FeatureWeight, axis=1))
                   
        return q 

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class TSClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, timesteps, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
        dist_metric: distance metric between sequences used in similarity kernel ('eucl', 'cir', 'cor' or 'acf').
    # Input shape
        3D tensor with shape: `(n_samples, timesteps, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, dist_metric='eucl', **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(TSClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.dist_metric = dist_metric
        self.initial_weights = weights
        self.clusters = None
        self.built = False

    def build(self, input_shape):
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_shape[1]))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_shape[1]), initializer='glorot_uniform', name='cluster_centers')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    '''def call(self, inputs, **kwargs):
        """
        Student t-distribution kernel, probability of assigning encoded sequence i to cluster k.
            q_{ik} = (1 + dist(z_i, m_k)^2)^{-1} / normalization.

        Arguments:
            inputs: encoded input sequences, shape=(n_samples, timesteps, n_features)
        Return:
            q: soft labels for each sample. shape=(n_samples, n_clusters)
        """
        if self.dist_metric == 'eucl':
            distance = K.sum(K.sqrt(K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=1)), axis=-1)
        elif self.dist_metric == 'cid':
            ce_x = K.sqrt(K.sum(K.square(inputs[:, 1:, :] - inputs[:, :-1, :]), axis=1))  # shape (n_samples, n_features)
            ce_w = K.sqrt(K.sum(K.square(self.clusters[:, 1:, :] - self.clusters[:, :-1, :]), axis=1))  # shape (n_clusters, n_features)
            ce = K.maximum(K.expand_dims(ce_x, axis=1), ce_w) / K.minimum(K.expand_dims(ce_x, axis=1), ce_w)  # shape (n_samples, n_clusters, n_features)
            ed = K.sqrt(K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2))  # shape (n_samples, n_clusters, n_features)
            distance = K.sum(ed * ce, axis=-1)  # shape (n_samples, n_clusters)
        elif self.dist_metric == 'cor':
            inputs_norm = (inputs - K.expand_dims(K.mean(inputs, axis=1), axis=1)) / K.expand_dims(K.std(inputs, axis=1), axis=1)  # shape (n_samples, timesteps, n_features)
            clusters_norm = (self.clusters - K.expand_dims(K.mean(self.clusters, axis=1), axis=1)) / K.expand_dims(K.std(self.clusters, axis=1), axis=1)  # shape (n_clusters, timesteps, n_features)
            pcc = K.mean(K.expand_dims(inputs_norm, axis=1) * clusters_norm, axis=2)  # Pearson correlation coefficients
            distance = K.sum(K.sqrt(2.0 * (1.0 - pcc)), axis=-1)  # correlation-based similarities, shape (n_samples, n_clusters)
        elif self.dist_metric == 'acf':
            raise NotImplementedError
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')
        q = 1.0 / (1.0 + K.square(distance) / self.alpha)
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q'''
    
    def call(self, inputs, **kwargs):
        """
        Student t-distribution kernel, probability of assigning encoded sequence i to cluster k.
            q_{ik} = (1 + dist(z_i, m_k)^2)^{-1} / normalization.

        Arguments:
            inputs: encoded input sequences, shape=(n_samples, n_features)
        Return:
            q: soft labels for each sample. shape=(n_samples, n_clusters)
        """
        inputs_expanded = K.expand_dims(inputs, axis=1)  # shape=(n_samples, 1, n_features)
        clusters_expanded = K.expand_dims(self.clusters, axis=0)  # shape=(1, n_clusters, n_features)

        if self.dist_metric == 'eucl':
            distance = K.sqrt(K.sum(K.square(inputs_expanded - clusters_expanded), axis=-1))
        elif self.dist_metric == 'cid':
            ce_x = K.sqrt(K.sum(K.square(inputs_expanded - K.expand_dims(inputs[:, :-1], axis=2)), axis=-1))  # shape (n_samples, timesteps)
            ce_w = K.sqrt(K.sum(K.square(clusters_expanded - K.expand_dims(self.clusters[:, :-1], axis=1)), axis=-1))  # shape (n_clusters, timesteps)
            ce = K.maximum(K.expand_dims(ce_x, axis=1), ce_w) / K.minimum(K.expand_dims(ce_x, axis=1), ce_w)  # shape (n_samples, n_clusters, timesteps)
            ed = K.sqrt(K.sum(K.square(inputs_expanded - clusters_expanded), axis=-1))  # shape (n_samples, n_clusters)
            distance = K.sum(ed * ce, axis=-1)  # shape (n_samples, n_clusters)
        elif self.dist_metric == 'cor':
            inputs_norm = (inputs - K.mean(inputs, axis=1, keepdims=True)) / K.std(inputs, axis=1, keepdims=True)  # shape (n_samples, n_features)
            clusters_norm = (self.clusters - K.mean(self.clusters, axis=1, keepdims=True)) / K.std(self.clusters, axis=1, keepdims=True)  # shape (n_clusters, n_features)
            pcc = K.mean(K.expand_dims(inputs_norm, axis=1) * K.expand_dims(clusters_norm, axis=0), axis=-1)  # Pearson correlation coefficients
            distance = K.sqrt(2.0 * (1.0 - pcc))  # correlation-based similarities, shape (n_samples, n_clusters)
        elif self.dist_metric == 'acf':
            raise NotImplementedError
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')

        q = 1.0 / (1.0 + K.square(distance) / self.alpha)
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q


    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters, 'dist_metric': self.dist_metric}
        base_config = super(TSClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# Function to save the class
import json
import numpy as np
def save_dtc_class(dtc, path=None):
    # Save the class variables
    #with open("dtc_state.json", "w") as state_file:
    #    json.dump(dtc.__dict__, state_file)

    # Save the Keras models
    if dtc.model:
        dtc.model.save_weights("dtc_model.h5")
    if dtc.autoencoder:
        dtc.autoencoder.save_weights("dtc_autoencoder.h5")
        dtc.autoencoderNeighbour.save_weights("dtc_autoencoder_neighbour.h5")
    if dtc.encoder:
        dtc.encoder.save_weights("dtc_encoder.h5")
        dtc.encoderNeighbour.save_weights("dtc_encoder_neighbour.h5")
    if dtc.decoder:
        dtc.decoder.save_weights("dtc_decoder.h5")
        dtc.decoderNeighbour.save_weights("dtc_decoder_neighbour.h5")
    print("DTC class saved successfully!")

In [ ]:
import numpy as np
import scipy.spatial.distance as distance
import scipy.sparse.linalg as linalg
import networkx as nx
import matplotlib.pyplot as plt
#from fastdtw import fastdtw

def complexity_estimate(series):
    """Computes the complexity estimate of a time series."""
    return np.sqrt(np.sum(np.diff(series) ** 2))

def cid_distance(x, y):
    """Computes the Complexity Invariant Distance (CID) between two time series."""
    euclidean_dist = np.linalg.norm(x - y)
    ce_x = complexity_estimate(x)
    ce_y = complexity_estimate(y)
    return euclidean_dist * ((ce_x + ce_y) / 2)

def compute_similarity_matrix(data, metric='euclidean', sigma=1.0):
    """
    Compute the adjacency (similarity) matrix using various distance metrics.
    """
    if metric in ['euclidean', 'cityblock', 'cosine', 'chebyshev']:
        pairwise_distances = distance.pdist(data, metric=metric)  # Compute pairwise distances
    elif metric == 'dtw':
        pairwise_distances = np.array([[fastdtw(data[i], data[j])[0] for j in range(len(data))] for i in range(len(data))])
    elif metric == 'cid':
        pairwise_distances = np.array([[cid_distance(data[i], data[j]) for j in range(len(data))] for i in range(len(data))])
    else:
        raise ValueError(f"Unsupported metric: {metric}")
    
    if metric in ['euclidean', 'cityblock', 'cosine', 'chebyshev']:
        adjacency_matrix = np.exp(-pairwise_distances ** 2 / (2. * sigma ** 2)) if metric == 'euclidean' else 1 / (1 + pairwise_distances)  # Apply transformation
        adjacency_matrix = distance.squareform(adjacency_matrix)  # Convert to square form
    else:
        adjacency_matrix = 1 / (1 + pairwise_distances)  # Normalize distances
    
    np.fill_diagonal(adjacency_matrix, 0)  # Remove self-loops
    return adjacency_matrix

def compute_graph_laplacian(adjacency_matrix, normalized=True):
    """
    Compute the graph Laplacian matrix.
    """
    degree_matrix = np.diag(np.sum(adjacency_matrix, axis=1))
    laplacian = degree_matrix - adjacency_matrix  # Unnormalized Laplacian
    
    if normalized:
        d_inv_sqrt = np.diag(1.0 / np.sqrt(np.diag(degree_matrix) + 1e-8))  # Avoid division by zero
        laplacian = np.matmul(np.matmul(d_inv_sqrt, laplacian), d_inv_sqrt)  # L_sym = D^(-1/2) * L * D^(-1/2)
    
    return laplacian

In [ ]:
class DTC:
    def __init__(self, n_clusters, input_dim, timesteps,
                 alpha=1.0, dist_metric='eucl', cluster_init='kmeans', heatmap=False):
        self.n_clusters = n_clusters
        self.input_dim = None
        self.timesteps = 791
        self.latent_shape = 10
        self.alpha = alpha
        self.dist_metric = dist_metric
        self.cluster_init = cluster_init
        self.heatmap = heatmap
        self.pretrained = False
        self.alpha2 = 0.8
        self.learning_rate = None
        self.optimizer = keras.optimizers.Adam()
        self.model = self.autoencoder = self.encoder = self.decoder = self.predmodel =  None
        self.autoencoderNeighbour = self.encoderNeighbour = self.decoderNeighbour =  None
        self.FeatureWeight = None
        self.a = 1.0
        self.b = 1.0
        self.initial_weights = None
        self.a_b = [1.93, 0.79]
        self.KNN = None
        self.b1 = -0.9
        self.b2 = self.b1/2

        
    def initialize(self):
        """
        Create DTC model
        """
        # Create AE models
        self.FeatureWeight = np.ones((self.n_clusters, self.latent_shape),dtype='float32')/self.latent_shape
        self.autoencoder, self.encoder, self.decoder = autoencoder_NN(series_len=144)
        self.autoencoderNeighbour , self.encoderNeighbour , self.decoderNeighbour = autoencoder_NN_Neighbour(series_len=144)
        clustering_layer = TSClusteringLayer(self.n_clusters,
                                             alpha=self.alpha,
                                             dist_metric=self.dist_metric,
                                             name='TSClustering')(self.encoderNeighbour.output)
        # Create DTC model
        self.model = Model(inputs=[self.autoencoderNeighbour.input],
                               outputs=[self.autoencoderNeighbour.output, clustering_layer])
        self.KNN = NearestNeighbors(n_neighbors=3,metric='euclidean')

    @property
    def cluster_centers_(self):
        """
        Returns cluster centers
        """
        return self.model.get_layer(name='TSClustering').get_weights()[0]
    
    def reconstruction_loss(self, data, reconstruction):
        # Mean squared error over each sample in the batch
        mse_loss = keras.losses.mean_squared_error(data, reconstruction)  # Shape: (267,)
        
        # Mean loss across the entire batch
        reconstruction_loss = tf.reduce_mean(mse_loss)  # Single scalar value
        return reconstruction_loss


    @staticmethod
    def weighted_kld(loss_weight):
        """
        Custom KL-divergence loss with a variable weight parameter
        """
        def loss(y_true, y_pred):
            return loss_weight * kullback_leibler_divergence(y_true, y_pred)
        return loss
    
    def loss_kld(self, y_true, y_pred):
        return kullback_leibler_divergence(y_true, y_pred)
    
    
    def loss_kld(self, y_true, y_pred):
        return kullback_leibler_divergence(y_true, y_pred)
    
    def categorical_cross_entropy(self, y_true, y_pred):
        cce = CategoricalCrossentropy()
        return cce(y_true, y_pred)
        
    def on_epoch_end(self, epoch):
        """
        Update heatmap loss weight on epoch end
        """
        if epoch > self.finetune_heatmap_at_epoch:
            K.set_value(self.heatmap_loss_weight, self.final_heatmap_loss_weight)

    def compile(self, gamma, optimizer, initial_heatmap_loss_weight=None, final_heatmap_loss_weight=None):
        """
        Compile DTC model

        # Arguments
            gamma: coefficient of TS clustering loss
            optimizer: optimization algorithm
            initial_heatmap_loss_weight (optional): initial weight of heatmap loss vs clustering loss
            final_heatmap_loss_weight (optional): final weight of heatmap loss vs clustering loss (heatmap finetuning)
        """
        if self.heatmap:
            self.initial_heatmap_loss_weight = initial_heatmap_loss_weight
            self.final_heatmap_loss_weight = final_heatmap_loss_weight
            self.heatmap_loss_weight = K.variable(self.initial_heatmap_loss_weight)
            self.model.compile(loss=['mse', DTC.weighted_kld(1.0 - self.heatmap_loss_weight), DTC.weighted_kld(self.heatmap_loss_weight)],
                               loss_weights=[1.0, gamma, gamma],
                               optimizer=optimizer)
        else:
            self.model.compile(loss=['mse', 'CategoricalCrossentropy'],
                               loss_weights=[1.0, gamma],
                               optimizer=optimizer)

    def load_weights(self, weights_path):
        """
        Load pre-trained weights of DTC model

        # Arguments
            weight_path: path to weights file (.h5)
        """
        self.model.load_weights(weights_path)
        self.pretrained = True

    def load_ae_weights(self, ae_weights_path):
        """
        Load pre-trained weights of AE

        # Arguments
            ae_weight_path: path to weights file (.h5)
        """
        self.autoencoder.load_weights(ae_weights_path)
        self.pretrained = True


    def manifold_loss(self, data, reconstruction, features):

        # Choose the similarity measure (example: Pearson)
        adj_matrix = compute_similarity_matrix(data, metric='cid')
        L = compute_graph_laplacian(adj_matrix, normalized=True)

        L = tf.convert_to_tensor(L, dtype=tf.float32)  # Convert sparse matrix to dense before TF tensor
        """Custom loss function combining reconstruction loss with whole-dataset manifold regularization."""
        
        reconstruction_loss = self.reconstruction_loss(data, reconstruction)
        
        
        # Compute manifold regularization using the entire dataset
        graph_loss_latent = tf.linalg.trace(tf.matmul(tf.matmul(tf.transpose(features), L), features))
        graph_loss_reconstruction = tf.linalg.trace(tf.matmul(tf.matmul(tf.transpose(reconstruction), L), reconstruction))
        print("Graph Loss: ", graph_loss_latent)
        return reconstruction_loss +  0.01 * graph_loss_latent # + 0.01 * graph_loss_reconstruction # Weighted sum
        
    def load_ae_neighbour_weights(self):
        input = self.autoencoder.get_layer(name='input').get_weights()
        encoder_0 = self.autoencoder.get_layer(name='Encoder').get_layer(name='encode_FC1_500').get_weights()
        encoder_1 = self.autoencoder.get_layer(name='Encoder').get_layer(name='encode_FC2_500').get_weights()
        encoder_2 = self.autoencoder.get_layer(name='Encoder').get_layer(name='encode_FC3_2000').get_weights()
        dense_encode = self.autoencoder.get_layer(name='Encoder').get_layer(name="latent_space").get_weights()
        decoder_0 = self.autoencoder.get_layer(name='Decoder').get_layer(name='decode_FC1_2000').get_weights()
        decoder_1 = self.autoencoder.get_layer(name='Decoder').get_layer(name='decode_FC2_500').get_weights()
        decoder_2 = self.autoencoder.get_layer(name='Decoder').get_layer(name='decode_FC3_500').get_weights()
        decoder_timeD = self.autoencoder.get_layer(name='Decoder').get_layer(name="reconstructed_output").get_weights()

        # Image weight
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='input').set_weights(input)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_FC1_500').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_FC2_500').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_FC3_2000').set_weights(encoder_2)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='latent_space').set_weights(dense_encode)
        
        

        
        self.autoencoderNeighbour.get_layer(name='Decoder').get_layer(name='decode_FC1_2000').set_weights(decoder_0)
        self.autoencoderNeighbour.get_layer(name='Decoder').get_layer(name='decode_FC2_500').set_weights(decoder_1)
        self.autoencoderNeighbour.get_layer(name='Decoder').get_layer(name='decode_FC3_500').set_weights(decoder_2)

        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour1_FC1_500').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour1_FC2_500').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour1_FC3_2000').set_weights(encoder_2)
        
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour2_FC1_500').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour2_FC2_500').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='Encoder').get_layer(name='encode_Neighbour2_FC3_2000').set_weights(encoder_2)

        self.autoencoderNeighbour.get_layer(name='Neighbour1_input').set_weights(input)
        self.autoencoderNeighbour.get_layer(name='Neighbour2_input').set_weights(input)

        self.encoderNeighbour.get_layer(name='input').set_weights(input)
        self.encoderNeighbour.get_layer(name='encode_FC1_500').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='encode_FC2_500').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='encode_FC3_2000').set_weights(encoder_2)
        
        self.encoderNeighbour.get_layer(name='encode_Neighbour1_FC1_500').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='encode_Neighbour1_FC2_500').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='encode_Neighbour1_FC3_2000').set_weights(encoder_2)
        
        self.encoderNeighbour.get_layer(name='encode_Neighbour2_FC1_500').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='encode_Neighbour2_FC2_500').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='encode_Neighbour2_FC3_2000').set_weights(encoder_2)

        self.encoderNeighbour.get_layer(name='Neighbour1_input').set_weights(input)
        self.encoderNeighbour.get_layer(name='Neighbour2_input').set_weights(input)

    def dist(self, x1, x2):
        """
        Compute distance between two multivariate time series using chosen distance metric

        # Arguments
            x1: first input (np array)
            x2: second input (np array)
        # Return
            distance
        """
        if self.dist_metric == 'eucl':
            return tsdistances.eucl(x1, x2)
        elif self.dist_metric == 'cid':
            return tsdistances.cid(x1, x2)
        elif self.dist_metric == 'cor':
            return tsdistances.cor(x1, x2)
        elif self.dist_metric == 'acf':
            return tsdistances.acf(x1, x2)
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')

    def init_cluster_weights(self, X):
        """
        Initialize with complete-linkage hierarchical clustering or k-means.

        # Arguments
            X: numpy array containing training set or batch
        """
        assert(self.cluster_init in ['hierarchical', 'kmeans'])
        print('Initializing cluster...')

        features = self.encode(X)

        if self.cluster_init == 'hierarchical':
            if self.dist_metric == 'eucl':  # use AgglomerativeClustering off-the-shelf
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='euclidean',
                                             linkage='complete').fit(features.reshape(features.shape[0], -1))
            else:  # compute distance matrix using dist
                d = np.zeros((features.shape[0], features.shape[0]))
                for i in range(features.shape[0]):
                    for j in range(i):
                        d[i, j] = d[j, i] = self.dist(features[i], features[j])
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='precomputed',
                                             linkage='complete').fit(d)
            # compute centroid
            cluster_centers = np.array([features[hc.labels_ == c].mean(axis=0) for c in range(self.n_clusters)])
        elif self.cluster_init == 'kmeans':
            # fit k-means on flattened features
            km = KMeans(n_clusters=self.n_clusters, n_init=10).fit(features.reshape(features.shape[0], -1))
            cluster_centers = km.cluster_centers_.reshape(self.n_clusters, features.shape[1])

        self.model.get_layer(name='TSClustering').set_weights([cluster_centers])
        print('Done!')

    def encode(self, x):
        """
        Encoding function. Extract latent features from hidden layer

        # Arguments
            x: data point
        # Return
            encoded (latent) data point
        """
        return self.encoder.predict(x)

    def decode(self, x):
        """
        Decoding function. Decodes encoded sequence from latent space.

        # Arguments
            x: encoded (latent) data point
        # Return
            decoded data point
        """
        return self.decoder.predict(x)

    def predict(self, x):
        """
        Predict cluster assignment.

        """
        q = self.model.predict(x, verbose=0)[1]
        return q.argmax(axis=1)
    
    @staticmethod
    def target_distribution(q):  # target distribution p which enhances the discrimination of soft label q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def predict_heatmap(self, x):
        """
        Produces TS clustering heatmap from input sequence.

        # Arguments
            x: data point
        # Return
            heatmap
        """
        return self.heatmap_model.predict(x, verbose=0)

    def pretrain(self, X,
                 optimizer='adam',
                 epochs=10,
                 batch_size=64,
                 save_dir='results/tmp',
                 verbose=1):
        """
        Pre-train the autoencoder using only MSE reconstruction loss
        Saves weights in h5 format.

        # Arguments
            X: training set
            optimizer: optimization algorithm
            epochs: number of pre-training epochs
            batch_size: training batch size
            save_dir: path to existing directory where weights will be saved
        """
        print('Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        # Begin pretraining
        #t0 = time()
        #self.autoencoder.fit(X, X, batch_size=batch_size, epochs=epochs)
        #print('Pretraining time: ', time() - t0)
        early_stopping = EarlyStopping(
        monitor='loss',  # Monitor the training loss
        patience=3,  # Number of epochs with no improvement before stopping
        restore_best_weights=True,  # Restore the best weights
        verbose=verbose
        )

        # Begin pretraining
        self.autoencoder.fit(
            X, X,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[early_stopping],  # Add EarlyStopping to callbacks
            verbose=verbose
        )
        self.autoencoder.save_weights('{}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        print('Pretrained weights are saved to {}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        self.pretrained = True

    def manifold_loss_pretrain(self, data, reconstruction):

        # Choose the similarity measure (example: Pearson)
        adj_matrix = compute_similarity_matrix(data, metric='cid')
        L = compute_graph_laplacian(adj_matrix, normalized=True)

        L = tf.convert_to_tensor(L, dtype=tf.float32)  # Convert sparse matrix to dense before TF tensor
        """Custom loss function combining reconstruction loss with whole-dataset manifold regularization."""
        
        reconstruction_loss = self.reconstruction_loss(data, reconstruction)
        
        
        # Compute manifold regularization using the entire dataset
        graph_loss_reconstruction = tf.linalg.trace(tf.matmul(tf.matmul(tf.transpose(reconstruction), L), reconstruction))
        return reconstruction_loss + 0.01 * graph_loss_reconstruction # Weighted sum

    
    def custom_fit(self, X_train, X_val=None, batch_size=10, epochs=200, patience=3, verbose=1, save_dir="/kaggle/working/"):
        optimizer = tf.keras.optimizers.Adam()
        loss_fn = tf.keras.losses.MeanSquaredError()
        
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, X_train)).batch(batch_size)
        
        best_train_loss = np.inf
        wait = 0
        
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            train_loss = []
            
            for step, (x_batch, y_batch) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    reconstructed = self.autoencoder(x_batch, training=True)
                    loss = self.manifold_loss_pretrain(y_batch, reconstructed)
                grads = tape.gradient(loss, self.autoencoder.trainable_variables)
                optimizer.apply_gradients(zip(grads, self.autoencoder.trainable_variables))
                train_loss.append(loss.numpy())
                
            avg_train_loss = np.mean(train_loss)
            print(f"  Train Loss: {avg_train_loss:.4f}")
            
            if avg_train_loss < best_train_loss:
                best_train_loss = avg_train_loss
                wait = 0
            else:
                wait += 1
                if wait >= patience:
                    print("Early stopping triggered on training loss.")
                    break


        self.autoencoder.save_weights('{}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        print('Pretrained weights are saved to {}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        self.pretrained = True
    
    def calculate_silhouette_score(self, data, labels):
        """
        Calculate the silhouette score for clustering results.

        Parameters:
        - data: np.array, the dataset with shape (n_samples, n_features)
        - labels: np.array, the cluster assignments for each sample

        Returns:
        - silhouette: float, the silhouette score ranging from -1 to 1
        """
        if len(np.unique(labels)) > 1:  # Silhouette score requires at least 2 clusters
            return silhouette_score(data, labels)
        else:
            return -1  # Return -1 if only one cluster exists
        
        
    
    def calculate_davies_bouldin_index(self, data, labels):
        """
        Calculate the Davies-Bouldin index for clustering results.

        Parameters:
        - data: np.array, the dataset with shape (n_samples, n_features)
        - labels: np.array, the cluster assignments for each sample

        Returns:
        - db_index: float, the Davies-Bouldin index (lower is better)
        """
        if len(np.unique(labels)) > 1:  # DBI requires at least 2 clusters
            return davies_bouldin_score(data, labels)
        else:
            return float('inf')  # Return infinity if only one cluster exists
    
    
    
    def calculate_calinski_harabasz_index(self, data, labels):
        """
        Calculate the Calinski-Harabasz index for clustering results.

        Parameters:
        - data: np.array, the dataset with shape (n_samples, n_features)
        - labels: np.array, the cluster assignments for each sample

        Returns:
        - ch_index: float, the Calinski-Harabasz index (higher is better)
        """
        if len(np.unique(labels)) > 1:  # Calinski-Harabasz requires at least 2 clusters
            return calinski_harabasz_score(data, labels)
        else:
            return 0  # Return 0 if only one cluster exists
    
    
    
    def save_metrics(self, metrics, file_path='metrics.csv'):
        """
        Save training metrics to a CSV file.

        # Arguments
            metrics: List of dictionaries containing metrics for each epoch
            file_path: Path to the CSV file where metrics will be saved
        """
        df = pd.DataFrame(metrics)
        df.to_csv(file_path, index=False)
        print(f'Metrics saved to {file_path}')
        
        
    def adjusted_rand_index(self, true_labels, y_pred):
        """
        Calculate the Adjusted Rand Index (ARI) between true and predicted labels.
        
        # Arguments
            true_labels: True cluster labels
            predicted_labels: Predicted cluster labels
        
        # Return
            ARI score
        """
        print(y_pred)
        return adjusted_rand_score(true_labels, y_pred)
    
    def normalized_mutual_info(self, true_labels, y_pred):
        """
        Calculate the Normalized Mutual Information (NMI) between true and predicted labels.
        
        # Arguments
            true_labels: True cluster labels
            predicted_labels: Predicted cluster labels
        
        # Return
            NMI score
        """
        print(y_pred)
        return normalized_mutual_info_score(true_labels, y_pred)


    def calculate_cid(self, x, y):
        """
        Calculate Complexity-Invariant Distance (CID) between two time series.
        """
        # Ensure the input time series are 1D
        x = np.squeeze(x)
        y = np.squeeze(y)
        
        # Compute Euclidean distance
        ed = np.sqrt(np.sum((x - y) ** 2))
        
        # Compute complexity of x
        complexity_x = np.sqrt(np.sum(np.diff(x) ** 2))
        
        # Compute complexity of y
        complexity_y = np.sqrt(np.sum(np.diff(y) ** 2))
        
        # Compute complexity ratio
        complexity_ratio = max(complexity_x, complexity_y) / min(complexity_x, complexity_y)
        
        # Calculate CID
        cid = ed * complexity_ratio
        return cid

    def calculate_cid_matrix(self, data):
        """
        Calculate a CID distance matrix for all pairs of time series in a dataset.
        
        Parameters:
            data (np.ndarray): 2D array where each row is a time series (n_series, series_length).
            
        Returns:
            np.ndarray: Distance matrix of shape (n_series, n_series).
        """
        n_series = data.shape[0]
        cid_matrix = np.zeros((n_series, n_series))
        
        for i in range(n_series):
            for j in range(i, n_series):  # Avoid duplicate calculations
                cid = self.calculate_cid(data[i], data[j])
                cid_matrix[i, j] = cid
                cid_matrix[j, i] = cid  # Symmetric matrix
        
        return cid_matrix

    def calculate_cid_silhouette_score(self, distance_matrix, labels):
        """
        Calculate silhouette score using a precomputed distance matrix.
    
        Parameters:
            distance_matrix (np.ndarray): Precomputed distance matrix (n_samples, n_samples).
            labels (np.ndarray): Cluster labels for each point (n_samples,).
    
        Returns:
            float: Silhouette score.
        """
        # Use the precomputed distance matrix with the 'precomputed' metric
        score = silhouette_score(distance_matrix, labels, metric="precomputed")
        return score


    def evaluate_clustering(self, data, y_pred, y_true):
        """
        Evaluate the clustering performance using ARI and NMI scores.
        
        # Arguments
            X: Data to be clustered
            y_true: True cluster labels
        
        # Return
            None
        """
        
        # Compute ARI and NMI
        ari_score = self.adjusted_rand_index(y_true, y_pred)
        nmi_score = self.normalized_mutual_info(y_true, y_pred)
        cid_distance = self.calculate_cid_matrix(data)
        silhouette_cid = self.calculate_cid_silhouette_score(cid_distance, y_pred)
        accuracy = cluster_acc(y_true, y_pred)
        purity = cluster_purity(y_true, y_pred)
        metrics = cluster_metrics(y_true, y_pred)
        
        print(f"Adjusted Rand Index (ARI): {ari_score:.4f}")
        print(f"Normalized Mutual Information (NMI): {nmi_score:.4f}")
        print(f"CID Silhouette Score: ", silhouette_cid)
        print(f"Accuracy: ", accuracy)
        print(f"Cluster Purity: ", purity)
        print(f"Accuracy: ", metrics['accuracy'])
        print(f"Precision: ", metrics['precision'])
        print(f"Recall: ", metrics['recall'])
        print(f"Rand Index: ", metrics['rand_index'])
        print(f"Fowlkes Mallows Index: ", metrics['fowlkes_mallows_index'])
    
    def fit(self, X_train, y_train=None, X_val=None, y_val=None,
        epochs=50,
        eval_epochs=10,
        save_epochs=10,
        batch_size=64,
        tol=0.001,
        patience=5,
        finetune_heatmap_at_epoch=8,
        save_dir='results/tmp',
        mutation_rate=0.01,
        crossover_rate=0.5,
        num_iterations=5):
            
            index = 0
            if not self.pretrained:
                print('Autoencoder was not pre-trained!')
            print('Training for {} epochs.\nEvaluating every {} and saving model every {} epochs.'.format(epochs, eval_epochs, save_epochs))
            best_val_error = float('inf')
            index_array = np.arange(X_train.shape[0])
            for epoch in range(epochs):
                if epoch == 0:
                    q = self.model.predict([X_train, X_train, X_train])[1]
                    extract = Model(inputs = self.model.input, outputs = self.model.get_layer('add_inputs').output)
                    z = extract.predict([X_train, X_train, X_train])
                else:
                    self.KNN.fit(z)
                    _,Neighbors_list=self.KNN.kneighbors(z)
                    q = self.model.predict([X_train,X_train[Neighbors_list[:,1]], X_train[Neighbors_list[:,2]]])[1]

                p = DTC.target_distribution(q)
                p_pred = p.argmax(axis=1)
                new_clustering = p_pred.copy()
                reliableindex, _ = np.unique(np.unique(np.where(np.sort(q,axis=1)[:,-1] - np.sort(q,axis=1)[:,-2]>=self.b2), np.where(np.max(q,axis=1)>=self.b1)),
                           np.unique(np.where(np.sort(p,axis=1)[:,-1] - np.sort(p,axis=1)[:,-2]>=self.b2), np.where(np.max(p,axis=1)>=self.b1)))

                print('Number of reliable samples:', len(q[reliableindex]) )
                print('Number of unreliable samples:', len(q)-len(q[reliableindex]) )

                Number_Unreliable = len(q)-len(q[reliableindex])
                Number_reliable = len(q[reliableindex])
                with tf.GradientTape() as tape:
                    idx = index_array[index:reliableindex.shape[0]]
                    self.KNN.fit(z[reliableindex[idx]])
                    _,Neighbors_list=self.KNN.kneighbors(z[reliableindex[idx]])
                    outputs = self.model([X_train[reliableindex[idx]], X_train[Neighbors_list[:,1]], X_train[Neighbors_list[:,2]]], training=True)
                    feature_extraction = Model(inputs = self.model.input, outputs = self.model.get_layer('add_inputs').output)
                    features = feature_extraction.predict([X_train[reliableindex[idx]], X_train[Neighbors_list[:,1]], X_train[Neighbors_list[:,2]]])
                    reconstruction_output, clustering_output = outputs
                    reconstruction_loss = self.manifold_loss(X_train[reliableindex[idx]], reconstruction_output, features)
                    clustering_loss = self.categorical_cross_entropy(p[reliableindex[idx]], clustering_output)

                    total_loss = tf.reduce_sum(reconstruction_loss) + tf.reduce_sum(clustering_loss)
                    total_loss = tf.reduce_sum(total_loss)


                feature_extraction = Model(inputs = self.model.input, outputs = self.model.get_layer('add_inputs').output)
                features = feature_extraction.predict([X_train, X_train, X_train])
                np.save(f"features_{epoch}.npy", features)
                print(f"Features saved to 'features_{epoch}.npy'")
                grads = tape.gradient(total_loss, self.model.trainable_variables)
                self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                print("Reconstruction Loss: ", reconstruction_loss)
                print("Clustering Loss: ", clustering_loss)
                num_elements = len(reliableindex[idx])
                n1 = Neighbors_list[:,1]
                n1 = n1[:num_elements]
                n2 = Neighbors_list[:,2]
                n2 = n2[:num_elements]
                z[idx] = extract.predict([X_train[reliableindex[idx]], X_train[n1], X_train[n2]])
                self.evaluate_clustering(X_train, np.array(p_pred), y_train)
                print("********************************************************************************")

In [ ]:
import pandas as pd

def load_data(train_path, test_path):
    # Load the train dataset
    train_df = pd.read_csv(train_path, delim_whitespace=True, header=None)
    
    # Load the test dataset
    test_df = pd.read_csv(test_path, delim_whitespace=True, header=None)
    
    # Extract features and labels for the train set
    X_train = train_df.iloc[:, 1:]  # All rows, all columns except the first one
    y_train = train_df.iloc[:, 0]   # All rows, first column
    
    # Extract features and labels for the test set
    X_test = test_df.iloc[:, 1:]  # All rows, all columns except the first one
    y_test = test_df.iloc[:, 0]   # All rows, first column
    
    return X_train, y_train, X_test, y_test

# Example usage
train_path = '/kaggle/input/timeseriesclassificationdataset/PowerCons/PowerCons_TRAIN.txt'
test_path = '/kaggle/input/timeseriesclassificationdataset/PowerCons/PowerCons_TEST.txt'

X_train, y_train, X_test, y_test = load_data(train_path, test_path)

X_train = X_train.values
X_test = X_test.values

print("Train features shape:", X_train.shape)
print("Train labels shape:", y_train.shape)
print("Test features shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

In [ ]:
print("X train shape: ",X_train.shape)
print("X test shape: ", X_test.shape)

In [ ]:
# Instantiate model
dtc = DTC(n_clusters=3,input_dim=X_train.shape[-1], timesteps=X_train.shape[1])

In [ ]:
optimizer = 'adam'
dtc.initialize()
dtc.model.summary()

In [ ]:
dtc.compile(gamma=1.0, optimizer=optimizer, initial_heatmap_loss_weight=0.1,
                final_heatmap_loss_weight=0.9)

In [ ]:
dtc.custom_fit(X_train)

In [ ]:
dtc.load_ae_neighbour_weights()

In [ ]:
# Initialize clusters
dtc.init_cluster_weights(X_train)

In [ ]:
# Fit model
#t0 = time()
dtc.fit(X_train, y_train=y_train-1, X_val=None, y_val=None)
#print('Training time: ', (time() - t0))

In [ ]:
# Evaluate
print('Performance (Test)')
results = {}
q = dtc.model.predict([X_test, X_test, X_test])[1]
y_pred = q.argmax(axis=1)

In [ ]:
dtc.evaluate_clustering(X_test, y_pred, y_test)

In [ ]:
save_dtc_class(dtc, path=None)